In [1]:
print(12)

12


In [55]:
from dotenv import load_dotenv
import os
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
#os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
#GOOGLE_API_KEY

In [27]:
import google.generativeai as genai

In [ ]:
from google import genai

In [28]:
for m in genai.list_models():
        print(f"Model name: {m.name}")

Model name: models/embedding-gecko-001
Model name: models/gemini-2.5-pro-preview-03-25
Model name: models/gemini-2.5-flash-preview-05-20
Model name: models/gemini-2.5-flash
Model name: models/gemini-2.5-flash-lite-preview-06-17
Model name: models/gemini-2.5-pro-preview-05-06
Model name: models/gemini-2.5-pro-preview-06-05
Model name: models/gemini-2.5-pro
Model name: models/gemini-2.0-flash-exp
Model name: models/gemini-2.0-flash
Model name: models/gemini-2.0-flash-001
Model name: models/gemini-2.0-flash-exp-image-generation
Model name: models/gemini-2.0-flash-lite-001
Model name: models/gemini-2.0-flash-lite
Model name: models/gemini-2.0-flash-preview-image-generation
Model name: models/gemini-2.0-flash-lite-preview-02-05
Model name: models/gemini-2.0-flash-lite-preview
Model name: models/gemini-2.0-pro-exp
Model name: models/gemini-2.0-pro-exp-02-05
Model name: models/gemini-exp-1206
Model name: models/gemini-2.0-flash-thinking-exp-01-21
Model name: models/gemini-2.0-flash-thinking-e

In [22]:
client = genai.Client()

# 3. Define the text you want to embed
text_to_embed = ["Hello world"]

# 4. Generate the embedding
result = client.models.embed_content(
    model="gemini-embedding-001",
    contents=text_to_embed
)

# 5. The result contains the embedding
print(result.embeddings)


[ContentEmbedding(
  values=[
    -0.02342152,
    0.01676572,
    0.009261323,
    -0.06383,
    -0.0026262768,
    <... 3067 more items ...>,
  ]
)]


In [26]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [39]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_youtube_transcript(video_url):
        try:
            # Extract the video ID from the URL
            video_id = video_url.split("v=")[1].split("&")[0]
            ytt_api = YouTubeTranscriptApi()

            # Fetch the transcript
            transcript_list = ytt_api.fetch(video_id)
            

            # Concatenate the text from each segment
            full_transcript = " ".join([segment.text for segment in transcript_list])
            return full_transcript
        except Exception as e:
            return f"Error: {e}"

    # Example usage:
video_link = " " # Replace with your desired YouTube video URL
transcript = get_youtube_transcript(video_link)

if "Error" not in transcript:
        print("Transcript obtained successfully:")
        print(transcript)
else:
        print("Transcript not obtained successfully:")
        print(transcript)

Transcript not obtained successfully:
Error: list index out of range


In [44]:
def get_youtube_transcript(video_url):
    # Extract the video ID from the URL
    full_transcript = "0"
    if not video_url or video_url == " ":
        return full_transcript
    video_id = video_url.split("v=")[1].split("&")[0]
    ytt_api = YouTubeTranscriptApi()

    # Fetch the transcript
    transcript_list = ytt_api.fetch(video_id)
            
    print(transcript_list)
    # Concatenate the text from each segment

    full_transcript = " ".join([segment.text for segment in transcript_list])
    return full_transcript



video_link = "https://www.youtube.com/watch?v=SCZ0BZq-jqY" # Replace with your desired YouTube video URL
transcript = get_youtube_transcript(video_link)
print("Transcript obtained successfully:")
print(transcript)

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='hello guys my name is B hammed and', start=0.44, duration=4.399), FetchedTranscriptSnippet(text='welcome to my course so guys in this', start=2.76, duration=3.999), FetchedTranscriptSnippet(text="video we'll be implementing one end to", start=4.839, duration=4.361), FetchedTranscriptSnippet(text='end genbi project called ENT medical', start=6.759, duration=4.601), FetchedTranscriptSnippet(text='chatbot so I think you already know what', start=9.2, duration=4.08), FetchedTranscriptSnippet(text="is medical chatbot let's say if you have", start=11.36, duration=4.199), FetchedTranscriptSnippet(text="any medical related query let's say you", start=13.28, duration=4.759), FetchedTranscriptSnippet(text='got one disease so you are asking for', start=15.559, duration=4.041), FetchedTranscriptSnippet(text='the diagnosis you are asking for the', start=18.039, duration=4.521), FetchedTranscriptSnippet(text='medicine so this bot will able t

In [45]:
video_link = " " # Replace with your desired YouTube video URL
transcript = get_youtube_transcript(video_link)
print("Transcript obtained successfully:")
print(transcript)

Transcript obtained successfully:
0


In [ ]:
import streamlit as st
from src.helper import get_pdf_text, get_youtube_transcript, get_text_chunks, get_vector_store, get_conversational_chain


def user_input(user_question):
    response = st.session_state.conversation({'question':user_question})
    st.session_state.chatHistory = response['chat_history']
    for i, message in enumerate(st.session_state.chatHistory):
        if i%2==0:
            st.write("User:", message.content)
        else:
            st.write("Reply:", message.content)

def main():
    st.set_page_config("Information Retrieval")
    st.header("Information Retrieval System")

    user_question = st.text_input("Ask a Question from PDF files")

    if "conversation" not in st.session_state:
        st.session_state.conversation = None
    if "chatHistory" not in st.session_state:
        st.session_state.chatHistory = None
    
    if user_question:
        user_input(user_question)

    with st.sidebar:
        st.title("Menu:")
        pdf_docs = st.file_uploader("Upload your PDF files here", accept_multiple_files=True)
        st.subheader("OR")
        yt_link = st.text_input("Enter your youtube link here")
        st.write("Click submit & process button once entered")
        if st.button("Submit & Process"):
            with st.spinner("Processing..."):
                raw_text_pdf = get_pdf_text(pdf_docs)
                raw_text_yt = get_youtube_transcript(yt_link)
                raw_text = "PDFs: "+ raw_text_pdf + "Youtube video transcript: " + raw_text_yt
                text_chunks = get_text_chunks(raw_text)
                vector_store = get_vector_store(text_chunks)
                st.session_state.conversation = get_conversational_chain(vector_store)
                st.success("Done")


if __name__ == "__main__":
    main()

In [52]:
text = "abcabc"
text = text.split("|")

In [54]:
def get_youtube_transcript(video_urls):
    # Extract the video ID from the URL
    full_transcript = ""
    if not video_urls or video_urls == " ":
        return full_transcript
    
    video_url_list = video_urls.split('|')

    for video_url in video_url_list:    
        video_id = video_url.split("v=")[1].split("&")[0]
        ytt_api = YouTubeTranscriptApi()

        # Fetch the transcript
        transcript_list = ytt_api.fetch(video_id)
            
        #print(transcript_list)
    # Concatenate the text from each segment

        full_transcript_temp = " ".join([segment.text for segment in transcript_list])
        full_transcript += full_transcript_temp
    return full_transcript

url = "https://www.youtube.com/watch?v=SCZ0BZq-jqY|https://www.youtube.com/watch?v=JYdd1k44FRM"
get_youtube_transcript(url)

"hello guys my name is B hammed and welcome to my course so guys in this video we'll be implementing one end to end genbi project called ENT medical chatbot so I think you already know what is medical chatbot let's say if you have any medical related query let's say you got one disease so you are asking for the diagnosis you are asking for the medicine so this bot will able to give the response with respect to that that means it will give you the suggestion it will give you the medicine suggestion it will give you the diagnosis suggestion so these kinds of information this bot will provide so we'll be creating the complete system here that means here first of all we'll be using our custom data the entire let's say medical information custom data we'll be using first of all we'll be teaching our model we'll be creating the entire knowledge base then I'm going to connect my large language model there that means whatever information you'll be asking related the medical let's say any kinds

In [ ]:
import streamlit as st
from src.helper import get_pdf_text, get_youtube_transcript, get_text_chunks, get_vector_store, get_conversational_chain


def user_input(user_question):
    response = st.session_state.conversation_0({'question':user_question})
    st.session_state.chatHistory_0 = response['chat_history']
    

def main():
    st.set_page_config("Information Retrieval")
    st.title("PDF Chatbot")
    if "conversation_0" not in st.session_state:
        st.session_state.conversation_0 = None
    if "chatHistory_0" not in st.session_state:
        st.session_state.chatHistory_0 = None
    
    if not st.session_state.conversation_0:
        with st.container(border=True):
            pdf_docs = st.file_uploader("Upload your PDF files here", accept_multiple_files=True)
            st.write("Click submit & process button once entered")
            if st.button("Submit & Process"):
                with st.spinner("Processing..."):
                    raw_text = get_pdf_text(pdf_docs)
                    text_chunks = get_text_chunks(raw_text)
                    vector_store = get_vector_store(text_chunks)
                    st.session_state.conversation_0 = get_conversational_chain(vector_store)
                    st.success("Done")

    user_question = st.chat_input("Ask a Question from PDF files")
    if user_question:
        user_input(user_question)

    if st.session_state.chatHistory_0:
        st.success("File(s) is in record!!")
        for i, message in enumerate(st.session_state.chatHistory_0):
            if i%2==0:
                st.write("User:", message.content)
            else:
                st.write("Reply:", message.content)


if __name__ == "__main__":
    main()